# Installing and Loading Libraries

In [1]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Creating Function

In [2]:
import pandas as  pd

In [3]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
def result_to_dataframe(result):
    
    if result == []:
        return 'No records Found'
    
    else:
    
        data = { column:[] for column, _ in result[0].items()}

        for record in result:

            for col, val in record.items():

                data[col].append(val)

        return pd.DataFrame(data)

In [14]:
db =  Neo4jConnection("bolt://localhost:7687", "neo4j", "Indira@6551")

# Query 1 Person role in that Movie

In [15]:
q  = '''match (p:Person) -[r]-> (m:Movie)
        return p.name as person ,type(r) as role ,m.title as movie'''

result = db.query(q)

result_to_dataframe(result)

,person,role,movie
0,Keanu Reeves,ACTED_IN,The Matrix
1,Carrie-Anne Moss,ACTED_IN,The Matrix
2,Laurence Fishburne,ACTED_IN,The Matrix
3,Hugo Weaving,ACTED_IN,The Matrix
4,Emil Eifrem,ACTED_IN,The Matrix
...,...,...,...
752,Woo-sik Choi,ACTED_IN,Parasite
753,So-dam Park,ACTED_IN,Parasite
754,Joaquin Phoenix,ACTED_IN,Joker
755,Robert De Niro,ACTED_IN,Joker


# Query 2 : List of Movie 

In [16]:
q  = '''match (m:Movie)
        return m.title as movie, m.tagline as tagline,m.released as released
        order by released desc'''

result = db.query(q)
result_to_dataframe(result)

,movie,tagline,released
0,Parasite,Act like you own the place,2019
1,Joker,Put on a happy face,2019
2,Parasite,Act like you own the place,2019
3,Joker,Put on a happy face,2019
4,Cloud Atlas,Everything is connected,2012
...,...,...,...
113,Top Gun,"I feel the need, the need for speed.",1986
114,Stand By Me,"For some, it's the last real taste of innocenc...",1986
115,One Flew Over the Cuckoo's Nest,"If he's crazy, what does that make you?",1975
116,One Flew Over the Cuckoo's Nest,"If he's crazy, what does that make you?",1975


# Query 3 : List of Person  along with roles

In [17]:
q  = '''match (p:Person) -[r] -> (m:Movie)
        return p.name as person , p.born as birth_year, type(r) as roles'''

result = db.query(q)
result_to_dataframe(result)

,person,birth_year,roles
0,Keanu Reeves,1964.0,ACTED_IN
1,Carrie-Anne Moss,1967.0,ACTED_IN
2,Laurence Fishburne,1961.0,ACTED_IN
3,Hugo Weaving,1960.0,ACTED_IN
4,Emil Eifrem,1978.0,ACTED_IN
...,...,...,...
752,Woo-sik Choi,NaN,ACTED_IN
753,So-dam Park,NaN,ACTED_IN
754,Joaquin Phoenix,NaN,ACTED_IN
755,Robert De Niro,NaN,ACTED_IN


# Query 4 : Top 5 People who worked most Number of movies as Director

In [18]:
q  = '''match (p) -[:DIRECTED]- (m)
        with p ,count(m) as no_of_movies
        return p.name as Person, no_of_movies
        order by no_of_movies desc
        Limit 5'''

result = db.query(q)
result_to_dataframe(result)

,Person,no_of_movies
0,Andy Wachowski,5
1,Lana Wachowski,5
2,Andy Wachowski,5
3,Lana Wachowski,5
4,Andy Wachowski,5


# Query 5: List of Person who worked as Director and Producerd in Same Movie
# ![Picture](Pictures/ql.png)

In [19]:
q  = '''match(p:Person) -[:DIRECTED]-> (m:Movie)
        match(p) -[:PRODUCED] -> (m)
        return p.name as person, p.born as birth_year, m.title as movie, m.released'''

result = db.query(q)
result_to_dataframe(result)

,person,birth_year,movie,m.released
0,Rob Reiner,1947,When Harry Met Sally,1998
1,Cameron Crowe,1957,Jerry Maguire,2000
2,Nancy Meyers,1949,Something's Gotta Give,2003
3,Rob Reiner,1947,When Harry Met Sally,1998
4,Cameron Crowe,1957,Jerry Maguire,2000
5,Nancy Meyers,1949,Something's Gotta Give,2003
6,Rob Reiner,1947,When Harry Met Sally,1998
7,Cameron Crowe,1957,Jerry Maguire,2000
8,Nancy Meyers,1949,Something's Gotta Give,2003


# Query 6: Person of worked more than 6 movies . Give names of person and movies he/ she worked

![Picture](Pictures/q6.png)

In [20]:
q =  """match (p:Person) --> (m:Movie)
        with p ,collect(m) as movies
        where size(movies) > 6
        UNWIND movies as m
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Keanu Reeves,The Matrix
1,Keanu Reeves,The Matrix Reloaded
2,Keanu Reeves,The Matrix Revolutions
3,Keanu Reeves,The Devil's Advocate
4,Keanu Reeves,The Replacements
...,...,...
109,Tom Hanks,Apollo 13
110,Tom Hanks,Cast Away
111,Tom Hanks,Charlie Wilson's War
112,Tom Hanks,The Polar Express


## Query 7 : Person who worked as writer and actor in same movie

![](Pictures/q7.png)

In [21]:
q =  """match (p:Person) -[:ACTED_IN]-> (m)
        match (p) -[:WROTE] -> (m)
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Aaron Sorkin,A Few Good Men
1,Aaron Sorkin,A Few Good Men
2,Aaron Sorkin,A Few Good Men


## Query 8 : Person who worked as director and actor in same movie

![](Pictures/q8.png)

In [22]:
q =  """match (p:Person) -[:ACTED_IN]-> (m)
        match (p) -[:DIRECTED] -> (m)
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Tom Hanks,That Thing You Do
1,Clint Eastwood,Unforgiven
2,Danny DeVito,Hoffa
3,Tom Hanks,That Thing You Do
4,Clint Eastwood,Unforgiven
5,Danny DeVito,Hoffa
6,Tom Hanks,That Thing You Do
7,Clint Eastwood,Unforgiven
8,Danny DeVito,Hoffa


# Query 9 Getting data to any Person Provide by User along work role in that movie:

For EG:
1. Lana Wachowski
2. Tom Hanks
3. Keanu Reeves

In [24]:
person =  input('Enter the name of Person: ')

q = f"""match (p:Person {{name:'{person}'}}) -[r]-> (m)
    return m.title as Movie , type(r) as Role"""

result = db.query(q)
result_to_dataframe(result)


,Movie,Role
0,You've Got Mail,ACTED_IN
1,Sleepless in Seattle,ACTED_IN
2,Joe Versus the Volcano,ACTED_IN
3,That Thing You Do,ACTED_IN
4,Cloud Atlas,ACTED_IN
5,The Da Vinci Code,ACTED_IN
6,The Green Mile,ACTED_IN
7,Apollo 13,ACTED_IN
8,Cast Away,ACTED_IN
9,Charlie Wilson's War,ACTED_IN


# Query 10 Getting data Person , Role and Movie which is/contains movie provide by User 

For EG:
1. The Matrix
2. The Matrix Reloaded
3. The Matrix Revolutions
4. Speed Racer
5. Cloud Atlas

In [25]:
movie = input('Enter the movie title which has or contains: ')

q = f"""match (p) -[r]-> (m:Movie)
    where m.title contains '{movie}'
    return p.name as Person,type(r) as Role ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Role,Movie
0,Hugo Weaving,ACTED_IN,Cloud Atlas
1,Tom Hanks,ACTED_IN,Cloud Atlas
2,Halle Berry,ACTED_IN,Cloud Atlas
3,Jim Broadbent,ACTED_IN,Cloud Atlas
4,Andy Wachowski,DIRECTED,Cloud Atlas
5,Lana Wachowski,DIRECTED,Cloud Atlas
6,Tom Tykwer,DIRECTED,Cloud Atlas
7,Jessica Thompson,REVIEWED,Cloud Atlas
8,Hugo Weaving,ACTED_IN,Cloud Atlas
9,Tom Hanks,ACTED_IN,Cloud Atlas


In [29]:
def graphs_to_csv(result):
    
    dct = {}
    
    for var in result:
        dct = { k[0]:{} for k in var.items() }
        break
        
    
    for i in result:
        d = i.items()
        for var , data in d:
            label = ' '.join(list(data.labels))
            dct[var][label] = {}
            properties = data._properties.keys()
            
            if not label in dct:
                dct[var][label] = { k : [] for k in properties }

            else:
                for i in properties:
                    if not i in dct[label]:
                        dct[var][label][i] = []
            
    for i in result:
        
        d = i.items()
        for var , data in d:
            label = ' '.join(list(data.labels))
            properties = data._properties
            
            for k in dct[var][label] :

                v = properties.get(k,float('nan'))

                dct[var][label][k].append(v)
    
    for var in dct:
        for label , v in dct[var].items():
            dct[var][label] = pd.DataFrame(v)
        
    return dct